## 非极大值抑制(NMS)及其优化实现

NMS的实现方法有：
1. 纯python实现
2. 直接利用Cython模块编译
3. 更改变量定义后再利用Cython模块编译
4. 在方法3的基础上利用gpu

NMS的优化：
1. Soft-NMS
2. 局部感知NMS
3. 倾斜NMS
4. 多边形NMS
5. 掩膜NMS

### NMS实现方式

纯python实现

In [ ]:
import numpy as np

def nms_py(dets, thresh):
    # same category
    # dets:(m,5)  thresh:scaler
    
    x1 = dets[:,0]
    y1 = dets[:,1]
    x2 = dets[:,2]
    y2 = dets[:,3]
    
    areas = (y2-y1+1) * (x2-x1+1)
    scores = dets[:,4]
    keep = []
    # 按照score从大到小排序
    index = scores.argsort()[::-1]
    
    while index.size >0:

        i = index[0]       # every time the first is the biggst, and add it directly
        keep.append(i)
        
        x11 = np.maximum(x1[i], x1[index[1:]])    # calculate the points of overlap 
        y11 = np.maximum(y1[i], y1[index[1:]])
        x22 = np.minimum(x2[i], x2[index[1:]])
        y22 = np.minimum(y2[i], y2[index[1:]])
        
        w = np.maximum(0, x22-x11+1)    # the weights of overlap
        h = np.maximum(0, y22-y11+1)    # the height of overlap
       
        overlaps = w*h
        
        ious = overlaps / (areas[i]+areas[index[1:]] - overlaps)
        
        idx = np.where(ious<=thresh)[0]
        
        index = index[idx+1]   # because index start from 1
        
    return keep

In [ ]:
# Test
import matplotlib.pyplot as plt

boxes = np.array(
    [
        [100, 100, 210, 210, 0.72],
        [250, 250, 420, 420, 0.8],
        [220, 220, 320, 330, 0.92],
        [100, 100, 210, 210, 0.72],
        [230, 240, 325, 330, 0.81],
        [220, 230, 315, 340, 0.9]
    ]
)

def plot_bbox(dets, c='k'):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    plt.plot([x1, x2], [y1, y1], c)
    plt.plot([x1, x1], [y1, y2], c)
    plt.plot([x1, x2], [y2, y2], c)
    plt.plot([x2, x2], [y1, y2], c)
    plt.title('after nms')

plot_bbox(boxes, 'k')
keep = nms_py(boxes, thresh=0.7)
plot_bbox(boxes[keep], 'r')

In [ ]:
# 测试运行效率
import time
import numpy as np

np.random.seed(1)
num_rois = 6000
minxy = np.random.randint(50, 145, size=(num_rois,2))
maxxy = np.random.randint(150, 200, size=(num_rois, 2))
score = 0.8*np.random.random_sample((num_rois, 1))+0.2

boxes_new = np.concatenate((minxy, maxxy, score), axis=1).astype(np.float32)

def nms_test_time(boxes_new):
    thresh = [0.7, 0.8, 0.9]
    T = 50
    for i in range(len(thresh)):
        since = time.time()
        for t in range(T):
            keep = nms_py(boxes_new, thresh=thresh[i])
        print("thresh={:.1f}, time cost:{:.4f}s".format(thresh[i], (time.time()-since)/T))
    # return keep

In [ ]:
nms_test_time(boxes_new)

### Cython模块编译

关于这一部分的内容可参考我的仓库：https://github.com/RyanCCC/Algorithm/tree/main/python

### Soft-NMS实现

论文：ICCV2017_Soft-NMS_Improving Object Detection With One Line of Code

官方实现：https://github.com/bharatsingh430/soft-nms

In [ ]:
def soft_nms(boxes, sigma, Nt, threshold, method=1):  # Nt为IOU阈值， threshold为最后得分的阈值
    N = boxes.shape[0]
    pos, maxpos, maxscore = 0, 0, 0

    for i in range(N):
        maxscore = boxes[i, 4]
        maxpos = i

        tx1 = boxes[i, 0]
        ty1 = boxes[i, 1]
        tx2 = boxes[i, 2]
        ty2 = boxes[i, 3]
        ts = boxes[i, 4]

        pos = i + 1

        # 获取最大位置，然后交换
        # 1. 获取最大分数位置
        while pos < N:
            if maxscore < boxes[pos, 4]:
                maxscore = boxes[pos, 4]
                maxpos = pos
            pos += 1

        # 2. 把最大分数位置换到i位置
        boxes[i, 0] = boxes[maxpos, 0]
        boxes[i, 1] = boxes[maxpos, 1]
        boxes[i, 2] = boxes[maxpos, 2]
        boxes[i, 3] = boxes[maxpos, 3]
        boxes[i, 4] = boxes[maxpos, 4]

        boxes[maxpos, 0] = tx1
        boxes[maxpos, 1] = ty1
        boxes[maxpos, 2] = tx2
        boxes[maxpos, 3] = ty2
        boxes[maxpos, 4] = ts

        # tx1, ty1, tx2, ty2, ts为最大分数的边框和置信度分数
        tx1 = boxes[i, 0]
        ty1 = boxes[i, 1]
        tx2 = boxes[i, 2]
        ty2 = boxes[i, 3]
        ts = boxes[i, 4]

        # 3. 重置pos位置，在步骤1中pos被置为N了
        pos = i + 1

        # NMS操作
        while pos < N:
            x1 = boxes[pos, 0]
            y1 = boxes[pos, 1]
            x2 = boxes[pos, 2]
            y2 = boxes[pos, 3]
            s = boxes[pos, 4]

            area = (x2 - x1 + 1) * (y2 - y1 + 1)
            iw = min(tx2, x2) - max(tx1, x1) + 1
            if iw > 0:
                ih = min(ty2, y2) - max(ty1, y1) + 1
                if ih > 0:
                    ovr = iw * ih / float((tx2 - tx1 + 1) * (ty2 - ty1 + 1) + area - iw * ih)

                    if method == 1:  # soft-NMS 线性
                        if ovr > Nt:
                            weight = 1 - ovr
                        else:
                            weight = 1
                    elif method == 2:  # soft-NMS 高斯
                        weight = np.exp(-(ovr * ovr) / sigma)
                    else:  # 原始NMS
                        if ovr > Nt:
                            weight = 0
                        else:
                            weight = 1

                    boxes[pos, 4] = weight * boxes[pos, 4]

                    # 如果不满足threshold阈值条件，则把最后面元素换到该位置，达到缩减boxes的目的
                    if boxes[pos, 4] < threshold:
                        boxes[pos, 0] = boxes[N - 1, 0]
                        boxes[pos, 1] = boxes[N - 1, 1]
                        boxes[pos, 2] = boxes[N - 1, 2]
                        boxes[pos, 3] = boxes[N - 1, 3]
                        boxes[pos, 4] = boxes[N - 1, 4]
                        N -= 1
                        pos -= 1
            pos += 1

    keep = [i for i in range(N)]
    return keep

In [ ]:
boxes = np.array([
        [204, 102, 358, 250, 0.5],
        [257, 118, 380, 250, 0.7],
        [280, 135, 400, 250, 0.6],
        [255, 118, 360, 235, 0.7]])
thresh = 0.3
threshold = 0.25
Nt = 0.3
sigma = 0.5
res = soft_nms(boxes, sigma, Nt, threshold)
print(boxes[res])

```python
# ----------------------------------------------------------
# Soft-NMS: Improving Object Detection With One Line of Code
# Copyright (c) University of Maryland, College Park
# Licensed under The MIT License [see LICENSE for details]
# Written by Navaneeth Bodla and Bharat Singh
# ----------------------------------------------------------
 
import numpy as np
cimport numpy as np
 
# 本文件是.pyx文件，是python的c扩展文件，要想被python调用、运行，仅仅写了源代码还是不够的，还要转成.c或者.c++的文件，并且再进一步转成.pyd文件；
# .pyd文件才是可以直接使用的文件，为了达到上述目的，就要写一个setup.py脚本，这个在nms文件夹中都有，就不专门介绍了；
 
# Cython是让Python脚本支持C语言扩展的编译器，Cython能够将Python+C混合编码的.pyx脚本转换为C代码，主要用于优化Python脚本性能或Python调用C函数库；
# 由于Python固有的性能差的问题，用C扩展Python成为提高Python性能常用方法，Cython算是较为常见的一种扩展方式；
 
 
# max函数
cdef inline np.float32_t max(np.float32_t a, np.float32_t b):
    return a if a >= b else b
 
# min函数
cdef inline np.float32_t min(np.float32_t a, np.float32_t b):
    return a if a <= b else b
 
# origin nms操作
def cpu_nms(np.ndarray[np.float32_t, ndim=2] dets, np.float thresh):
    cdef np.ndarray[np.float32_t, ndim=1] x1 = dets[:, 0]        # pred bbox top_x
    cdef np.ndarray[np.float32_t, ndim=1] y1 = dets[:, 1]        # pred bbox top_y
    cdef np.ndarray[np.float32_t, ndim=1] x2 = dets[:, 2]        # pred bbox bottom_x
    cdef np.ndarray[np.float32_t, ndim=1] y2 = dets[:, 3]        # pred bbox bottom_y
    cdef np.ndarray[np.float32_t, ndim=1] scores = dets[:, 4]    # pred bbox cls score
 
    cdef np.ndarray[np.float32_t, ndim=1] areas = (x2 - x1 + 1) * (y2 - y1 + 1)     # pred bbox areas
    cdef np.ndarray[np.int_t, ndim=1] order = scores.argsort()[::-1]                # 对pred bbox按score做降序排序，对应step-2
 
    cdef int ndets = dets.shape[0]                                                  # num of detected bbox
    cdef np.ndarray[np.int_t, ndim=1] suppressed = np.zeros((ndets), dtype=np.int)  # 相当于flag，与bbox对应，如果其已经在nms操作中被抑制(被认为与其他高score IoU过大，可剔除)，就置suppressed = 1，表示该bbox已经不纳入考虑
 
    cdef int _i, _j                               # nominal indices，和C的操作有点类似，先申明变量
    cdef int i, j                                 # sorted indices
    cdef np.float32_t ix1, iy1, ix2, iy2, iarea   # temp variables for box i's (the box currently under consideration)
    cdef np.float32_t xx1, yy1, xx2, yy2          # variables for computing overlap with box j (lower scoring box)
    cdef np.float32_t w, h
    cdef np.float32_t inter, ovr
 
    keep = []
    for _i in range(ndets):
        i = order[_i]            # 取当前index _i的score bbox，对应着此轮的最高score bbox
        if suppressed[i] == 1:   # 之前NMS操作已经被干掉了，无效bbox，那就忽略吧
            continue
        keep.append(i)           # 保留之
        ix1 = x1[i]
        iy1 = y1[i]
        ix2 = x2[i]
        iy2 = y2[i]
        iarea = areas[i]         # 面积
        for _j in range(_i + 1, ndets):    # 计算index _i的score bbox，与其之后bbox的IoU，进而做NMS
            j = order[_j]
            if suppressed[j] == 1:         # 无效bbox，忽略
                continue
            xx1 = max(ix1, x1[j])          # 为计算IoU做准备
            yy1 = max(iy1, y1[j])
            xx2 = min(ix2, x2[j])
            yy2 = min(iy2, y2[j])
            w = max(0.0, xx2 - xx1 + 1)    # Iinsection的宽、高、面积
            h = max(0.0, yy2 - yy1 + 1)
            inter = w * h
            ovr = inter / (iarea + areas[j] - inter)    # IoU
            if ovr >= thresh:         # 如果当前bbox与index _i的bbox，IoU过大，就要被抑制掉了
                suppressed[j] = 1
 
    return keep    # 最终NMS被保留的bbox
 
# soft_nms操作，这里假设boxes是无序(未按score做降序)的，所以每轮soft_nms迭代都需要类似冒泡排序操作，选择当前top-1 bbox做NMS
# Nt：计算IoU的阈值，IoU > Nt，对应bbox的score权重就要降低
# threshold：降权后通过threshold进一步剔除低权重bbox
def cpu_soft_nms(np.ndarray[float, ndim=2] boxes, float sigma=0.5, float Nt=0.3, float threshold=0.001, unsigned int method=0):
    cdef unsigned int N = boxes.shape[0]    # num of detected bbox
    cdef float iw, ih, box_area
    cdef float ua
    cdef int pos = 0
    cdef float maxscore = 0
    cdef int maxpos = 0
    cdef float x1,x2,y1,y2,tx1,tx2,ty1,ty2,ts,area,weight,ov
 
    for i in range(N):
        maxscore = boxes[i, 4]    # 获取当前index下的bbox
        maxpos = i
 
        tx1 = boxes[i,0]
        ty1 = boxes[i,1]
        tx2 = boxes[i,2]
        ty2 = boxes[i,3]
        ts = boxes[i,4]
 
        pos = i + 1      # 下面操作就很常规了，找到当前index i之后所有bboxes中，score最大的bbox，并将之赋值给maxscore、maxpos
        while pos < N:
            if maxscore < boxes[pos, 4]:
                maxscore = boxes[pos, 4]
                maxpos = pos
            pos = pos + 1
 
        # 下面操作更简单，想想我们最开始学C语言，a、b两变量如何交换
	    # add max box as a detection 
        boxes[i,0] = boxes[maxpos,0]    # maxpos内的信息，放到index i处，也是当前需要处理的bbox
        boxes[i,1] = boxes[maxpos,1]
        boxes[i,2] = boxes[maxpos,2]
        boxes[i,3] = boxes[maxpos,3]
        boxes[i,4] = boxes[maxpos,4]
 
	    # swap ith box with position of max box
        boxes[maxpos,0] = tx1           # 别忘了tx1中可是保存了boxes[i,0]备份的
        boxes[maxpos,1] = ty1
        boxes[maxpos,2] = tx2
        boxes[maxpos,3] = ty2
        boxes[maxpos,4] = ts
 
        tx1 = boxes[i,0]   # 此时tx1就保存的maxpos位置的bbox信息了
        ty1 = boxes[i,1]
        tx2 = boxes[i,2]
        ty2 = boxes[i,3]
        ts = boxes[i,4]
 
        pos = i + 1
	    # NMS iterations, note that N changes if detection boxes fall below threshold，N值是动态变化的
        while pos < N:     # 向后做NMS比较
            x1 = boxes[pos, 0]   # 当前位置的bbox
            y1 = boxes[pos, 1]
            x2 = boxes[pos, 2]
            y2 = boxes[pos, 3]
            s = boxes[pos, 4]
 
            area = (x2 - x1 + 1) * (y2 - y1 + 1)          # pos下box的面积
            iw = (min(tx2, x2) - max(tx1, x1) + 1)        # 计算Insection的宽iw，如果iw < 0，说明没相交，可以直接忽略了
            if iw > 0:
                ih = (min(ty2, y2) - max(ty1, y1) + 1)    # 计算Insection的宽ih，如果ih < 0，说明没相交，可以直接忽略了
                if ih > 0:
                    ua = float((tx2 - tx1 + 1) * (ty2 - ty1 + 1) + area - iw * ih)   # U的面积
                    ov = iw * ih / ua                                                # iou between max box and detection box
 
                    if method == 1:                       # soft_nms中linear降权操作，与ov负相关
                        if ov > Nt: 
                            weight = 1 - ov
                        else:
                            weight = 1
                    elif method == 2:                     # soft_nms中gaussian降权操作
                        weight = np.exp(-(ov * ov)/sigma)
                    else:                                 # original NMS，weight = 0就直接把score置0
                        if ov > Nt: 
                            weight = 0
                        else:
                            weight = 1
 
                    boxes[pos, 4] = weight * boxes[pos, 4]  # 权重重新调整
		    
		            # if box score falls below threshold, discard the box by swapping with last box，update N
                    # 如果bbox调整后的权重，已经小于阈值threshold，那么这个bbox就可以忽略了，
                    # 操作方式是直接用最后一个有效的bbox替换当前pos上的bbox
                    if boxes[pos, 4] < threshold:
                        boxes[pos,0] = boxes[N-1, 0]
                        boxes[pos,1] = boxes[N-1, 1]
                        boxes[pos,2] = boxes[N-1, 2]
                        boxes[pos,3] = boxes[N-1, 3]
                        boxes[pos,4] = boxes[N-1, 4]
                        N = N - 1           # N-1位置上的bbox已经赋值到前面了，该bbox就可以忽略了；
                        pos = pos - 1       # pos位置上引入了新的有效bbox(N-1)，就需要再计算一遍了
 
            pos = pos + 1 # 当前pos bbox计算完毕
 
    # 求满足soft_nms筛选条件的所有bbox数量，并打散为list，但一个问题是：如何与bbox index对应起来？
    # 方式很简单，bbox也做了对应的调整、筛选，bbox list中top-N就对应着最高score，且soft-nms筛选通过的bbox，
    # 不过每个bbox的score也同样经过soft-nms调整了
    keep = [i for i in range(N)]
 
    return keep

```

### 局部感知NMS（LNMS）

该变体在EAST文本检测中提出。主要原因是文本检测面临的是成千上万个几何体，如果用普通的NMS，其计算复杂度，n是几何体的个数，这是不可接受的。

LMNS基本步骤：
1. 先对所有的output box集合结合相应的阈值（大于阈值则进行合并，小于阈值则不和并）
2. 依次遍历进行加权合并，得到合并后的bbox集合
3. 对合并后的bbox集合进行标准NMS操作

In [ ]:
import numpy as np
from shapely.geometry import Polygon

def intersection(g, p):
    #取g,p中的几何体信息组成多边形
    g = Polygon(g[:8].reshape((4, 2)))
    p = Polygon(p[:8].reshape((4, 2)))

    # 判断g,p是否为有效的多边形几何体
    if not g.is_valid or not p.is_valid:
        return 0

    # 取两个几何体的交集和并集
    inter = Polygon(g).intersection(Polygon(p)).area
    union = g.area + p.area - inter
    if union == 0:
        return 0
    else:
        return inter/union

def weighted_merge(g, p):
    # 取g,p两个几何体的加权（权重根据对应的检测得分计算得到）
    g[:8] = (g[8] * g[:8] + p[8] * p[:8])/(g[8] + p[8])
    
    #合并后的几何体的得分为两个几何体得分的总和
    g[8] = (g[8] + p[8])
    return g

def standard_nms(S, thres):
    #标准NMS
    order = np.argsort(S[:, 8])[::-1]
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        ovr = np.array([intersection(S[i], S[t]) for t in order[1:]])
        inds = np.where(ovr <= thres)[0]
        order = order[inds+1]
        
    return S[keep]

def nms_locality(polys, thres=0.3):
    '''
    locality aware nms of EAST
    :param polys: a N*9 numpy array. first 8 coordinates, then prob
    :return: boxes after nms
    '''
    S = []    #合并后的几何体集合
    p = None   #合并后的几何体
    for g in polys:
        if p is not None and intersection(g, p) > thres:    #若两个几何体的相交面积大于指定的阈值，则进行合并
            p = weighted_merge(g, p)
        else:    #反之，则保留当前的几何体
            if p is not None:
                S.append(p)
            p = g
    if p is not None:
        S.append(p)
    if len(S) == 0:
        return np.array([])
    return standard_nms(np.array(S), thres)


In [ ]:
print(Polygon(np.array([[343, 350], [448, 135],[474, 143], [369, 359]])).area)

### 倾斜NMS

解决倾斜的文本行的检测。基本步骤如下：
1. 对输出的检测框rbox按照得分进行降序排序rbox_lists；
2. 依次遍历上述的rbox_lists．具体的做法是：将当前遍历的rbox与剩余的rbox进行交集运算得到相应的相交点集合，并根据判断相交点集合组成的凸边形的面积，计算每两个rbox的IOU；对于大于设定阈值的rbox进行滤除，保留小于设定阈值的rbox；
3. 得到最终的检测框

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import cv2
import tensorflow as tf

def nms_rotate(decode_boxes, scores, iou_threshold, max_output_size,
               use_angle_condition=False, angle_threshold=0, use_gpu=False, gpu_id=0):
    """
    :param boxes: format [x_c, y_c, w, h, theta]
    :param scores: scores of boxes
    :param threshold: iou threshold (0.7 or 0.5)
    :param max_output_size: max number of output
    :return: the remaining index of boxes
    """
    if use_gpu:
        #采用gpu方式
        keep = nms_rotate_gpu(boxes_list=decode_boxes,
                              scores=scores,
                              iou_threshold=iou_threshold,
                              angle_gap_threshold=angle_threshold,
                              use_angle_condition=use_angle_condition,
                              device_id=gpu_id)

        keep = tf.cond(
            tf.greater(tf.shape(keep)[0], max_output_size),
            true_fn=lambda: tf.slice(keep, [0], [max_output_size]),
            false_fn=lambda: keep)
    else: #采用cpu方式
        keep = tf.py_func(nms_rotate_cpu,
                          inp=[decode_boxes, scores, iou_threshold, max_output_size],
                          Tout=tf.int64)
    return keep

def nms_rotate_cpu(boxes, scores, iou_threshold, max_output_size):
    keep = []
    #保留框的结果集合
    order = scores.argsort()[::-1]
    #对检测结果得分进行降序排序
    num = boxes.shape[0]
    #获取检测框的个数

    suppressed = np.zeros((num), dtype=np.int)
    for _i in range(num):
        if len(keep) >= max_output_size:
            #若当前保留框集合中的个数大于max_output_size时，直接返回
            break

        i = order[_i]
        if suppressed[i] == 1:
            #对于抑制的检测框直接跳过
            continue
        keep.append(i)
        #保留当前框的索引
        r1 = ((boxes[i, 1], boxes[i, 0]), (boxes[i, 3], boxes[i, 2]), boxes[i, 4])  
        #根据box信息组合成opencv中的旋转bbox
        print("r1:{}".format(r1))
        area_r1 = boxes[i, 2] * boxes[i, 3]
        #计算当前检测框的面积
        for _j in range(_i + 1, num):
            #对剩余的而进行遍历
            j = order[_j]
            if suppressed[i] == 1:
                continue
            r2 = ((boxes[j, 1], boxes[j, 0]), (boxes[j, 3], boxes[j, 2]), boxes[j, 4])
            area_r2 = boxes[j, 2] * boxes[j, 3]
            inter = 0.0

            int_pts = cv2.rotatedRectangleIntersection(r1, r2)[1]
            #求两个旋转矩形的交集，并返回相交的点集合
            if int_pts is not None:
                order_pts = cv2.convexHull(int_pts, returnPoints=True)
                #求点集的凸边形
                int_area = cv2.contourArea(order_pts)
                #计算当前点集合组成的凸边形的面积
                inter = int_area * 1.0 / (area_r1 + area_r2 - int_area + 0.0000001)

            if inter >= iou_threshold:
                #对大于设定阈值的检测框进行滤除
                suppressed[j] = 1

    return np.array(keep, np.int64)

In [ ]:
boxes = np.array([[50, 40, 100, 100, 0],
                      [60, 50, 100, 100, 0],
                      [50, 30, 100, 100, -45.],
                      [200, 190, 100, 100, 0.]])

scores = np.array([0.99, 0.88, 0.66, 0.77])
keep = nms_rotate(tf.convert_to_tensor(boxes, dtype=tf.float32), tf.convert_to_tensor(scores, dtype=tf.float32),
                      0.7, 5)

### 多边形NMS（PNMS）

针对曲线文本提出的。

In [ ]:
import numpy as np
from shapely.geometry import *

def py_cpu_pnms(dets, thresh):
    # 获取检测坐标点及对应的得分
    bbox = dets[:, :4]
    scores = dets[:, 4]

    #这里文本的标注采用14个点，这里获取的是这14个点的偏移
    info_bbox = dets[:, 5:33]   

    #保存最终点坐标
    pts = []
    for i in range(dets.shape[0]):
        pts.append([[int(bbox[i, 0]) + info_bbox[i, j], int(bbox[i, 1]) + info_bbox[i, j+1]] for j in xrange(0,28,2)])

    areas = np.zeros(scores.shape)
    #得分降序
    order = scores.argsort()[::-1]
    inter_areas = np.zeros((scores.shape[0], scores.shape[0]))

    for il in range(len(pts)):
        #当前点集组成多边形，并计算该多边形的面积
        poly = Polygon(pts[il])
        areas[il] = poly.area
        
        #多剩余的进行遍历
        for jl in range(il, len(pts)):
            polyj = Polygon(pts[jl])
            
            #计算两个多边形的交集，并计算对应的面积
            inS = poly.intersection(polyj)
            inter_areas[il][jl] = inS.area
            inter_areas[jl][il] = inS.area

    #下面做法和nms一样
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        ovr = inter_areas[i][order[1:]] / (areas[i] + areas[order[1:]] - inter_areas[i][order[1:]])
        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]
        
    return keep

### 掩膜NMS（MNMS）

MNMS是在FTSN文本检测文章中提出的

In [28]:
import cv2
import numpy as np
import copy

EPS=0.00001

def get_mask(box,mask):
    """根据box获取对应的掩膜"""
    tmp_mask=np.zeros(mask.shape,dtype="uint8")
    tmp=np.array(box.tolist(),dtype=np.int32).reshape(-1,2)
    cv2.fillPoly(tmp_mask, [tmp], (255))
    tmp_mask=cv2.bitwise_and(tmp_mask,mask)
    return tmp_mask,cv2.countNonZero(tmp_mask)


def comput_mmi(area_a,area_b,intersect):
    """
    计算MMI,2018.11.23 add
    :param mask_a: 实例文本a的mask的面积
    :param mask_b: 实例文本b的mask的面积
    :param intersect: 实例文本a和实例文本b的相交面积
    :return:
    """
    if area_a==0 or area_b==0:
        area_a+=EPS
        area_b+=EPS
        print("the area of text is 0")
    return max(float(intersect)/area_a,float(intersect)/area_b)


def mask_nms(dets, mask, thres=0.3):
    """
    mask nms 实现函数
    :param dets: 检测结果，是一个N*9的numpy,
    :param mask: 当前检测的mask
    :param thres: 检测的阈值
    """
    # 获取bbox及对应的score
    bbox_infos=dets[:,:8]
    scores=dets[:,8]

    keep=[]
    order=scores.argsort()[::-1]
    print("order:{}".format(order))
    nums=len(bbox_infos)
    suppressed=np.zeros((nums), dtype=np.int)
    print("lens:{}".format(nums))

    # 循环遍历
    for i in range(nums):
        idx=order[i]
        if suppressed[idx]==1:
            continue
        keep.append(idx)
        mask_a,area_a=get_mask(bbox_infos[idx],mask)
        for j in range(i,nums):
            idx_j=order[j]
            if suppressed[idx_j]==1:
                continue
            mask_b, area_b =get_mask(bbox_infos[idx_j],mask)

            # 获取两个文本的相交面积
            merge_mask=cv2.bitwise_and(mask_a,mask_b)
            area_intersect=cv2.countNonZero(merge_mask)

            #计算MMI
            mmi=comput_mmi(area_a,area_b,area_intersect)
            # print("area_a:{},area_b:{},inte:{},mmi:{}".format(area_a,area_b,area_intersect,mmi))

            if mmi >= thres:
                suppressed[idx_j] = 1

    return dets[keep]

### 参考

1. [Python 非极大值抑制(NMS)的四种实现详解](https://www.wangyeyixia.com/article/1923856.html)
2. [目标检测之非极大值抑制(NMS)各种变体](https://zhuanlan.zhihu.com/p/50126479)
3. [手写NMS & soft-NMS](https://wzj.plus/2020/12/18/nms/)